In [1]:
# All imports
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import os.path
import json
import requests #To call API
from pandas import json_normalize
import re

In [2]:
# All unique cards
df_atomic = pd.read_csv("DataCleansed.csv",delimiter="|")

In [3]:
df_atomic

,Color,Subtype,Text,Type,Loyalty,Power,Toughness,Keyword,Name,ManaValue
0,['White'],['Plains'],({T}: Add {W}.),['Land'],NaN,NaN,NaN,['N/A'],Plains,0.0
1,['Blue'],['Island'],({T}: Add {U}.),['Land'],NaN,NaN,NaN,['N/A'],Island,0.0
2,['Black'],['Swamp'],({T}: Add {B}.),['Land'],NaN,NaN,NaN,['N/A'],Swamp,0.0
3,['Red'],['Mountain'],({T}: Add {R}.),['Land'],NaN,NaN,NaN,['N/A'],Mountain,0.0
4,['Green'],['Forest'],({T}: Add {G}.),['Land'],NaN,NaN,NaN,['N/A'],Forest,0.0
...,...,...,...,...,...,...,...,...,...,...
23919,['Colorless'],['N/A'],"White legendary creatures you control have ""ba...",['Land'],NaN,NaN,NaN,['N/A'],Cathedral of Serra,0.0
23920,['Colorless'],['N/A'],"Red legendary creatures you control have ""band...",['Land'],NaN,NaN,NaN,['N/A'],Mountain Stronghold,0.0
23921,['Colorless'],['N/A'],"Blue legendary creatures you control have ""ban...",['Land'],NaN,NaN,NaN,['N/A'],Seafarer's Quay,0.0
23922,['Blue'],['N/A'],{T}: Add {U}.\n{T}: Target creature loses band...,['Land'],NaN,NaN,NaN,['N/A'],Tolaria,0.0


In [4]:
names = list(df_atomic['Name'])

In [5]:
names

['Plains',
 'Island',
 'Swamp',
 'Mountain',
 'Forest',
 'Ajani, Caller of the Pride',
 "Ajani's Chosen",
 'Angelic Accord',
 'Angelic Wall',
 'Archangel of Thune',
 'Auramancer',
 'Banisher Priest',
 'Blessing',
 'Bonescythe Sliver',
 'Brave the Elements',
 'Capashen Knight',
 'Celestial Flare',
 'Charging Griffin',
 'Congregate',
 'Dawnstrike Paladin',
 'Devout Invocation',
 'Divine Favor',
 'Fiendslayer Paladin',
 'Fortify',
 'Griffin Sentinel',
 'Hive Stirrings',
 'Imposing Sovereign',
 'Indestructibility',
 'Master of Diversion',
 'Pacifism',
 'Path of Bravery',
 'Pay No Heed',
 'Pillarfield Ox',
 'Planar Cleansing',
 'Sentinel Sliver',
 'Seraph of the Sword',
 'Serra Angel',
 'Show of Valor',
 'Siege Mastodon',
 'Silence',
 'Solemn Offering',
 'Soulmender',
 'Steelform Sliver',
 'Stonehorn Chanter',
 'Suntail Hawk',
 'Wall of Swords',
 'Air Servant',
 'Archaeomancer',
 'Armored Cancrix',
 'Cancel',
 'Claustrophobia',
 'Clone',
 'Colossal Whale',
 'Coral Merfolk',
 'Dismiss into D

Get file with information about all the card printings: https://mtgjson.com/downloads/all-files/#allprintings 

In [5]:
# Open JSON file w/ data from all printings
file = open(r"./AllPrintings.json",encoding='utf-8')

# Transform from file to python dict
dictionary = json.load(file)

# Get only the card printing data
data_dict = dictionary['data']


In [6]:
def dict_to_df(data_dict):
    '''
    Transforms dictionary type data into a pandas dataframe.

            Parameters:
                    data_dict (dict): A dictionary

            Returns:
                    df (pandas.core.frame.DataFrame): A pandas dataframe with the data from data_dict
    '''
    for idx, key in enumerate(list(data_dict.keys())):
        if idx == 0: # If it's the first iteration, create df
            df_aux = json_normalize(data_dict[key]['cards']);
            df = df_aux
        else: # Else, append to df
            df_aux = json_normalize(data_dict[key]['cards'])
            df = pd.concat([df,df_aux], axis=0, ignore_index=True)

    return df

df = dict_to_df(data_dict)

In [7]:
df.head()

,colors,artist,layout,name,language,setCode,borderColor,colorIdentity,manaValue,convertedManaCost,...,isReserved,signature,isFunny,asciiName,isStorySpotlight,isTimeshifted,duelDeck,legalities.oldschool,hasContentWarning,cardParts
0,[],Adam Paquette,normal,Plains,Japanese,PMPS11,black,[W],0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],Adam Paquette,normal,Island,Japanese,PMPS11,black,[U],0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],Adam Paquette,normal,Swamp,Japanese,PMPS11,black,[B],0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],Adam Paquette,normal,Mountain,Japanese,PMPS11,black,[R],0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],Adam Paquette,normal,Forest,Japanese,PMPS11,black,[G],0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Save dataframe with printings information to file
df.to_csv("PrintingsData.csv","|",index=False)

In [8]:
# Open the file we just saved (so that we can comment cells 20:27 forever)
df = pd.read_csv("PrintingsData.csv",delimiter="|")

C:\Users\monic\AppData\Local\Temp\ipykernel_4608\2092063224.py:2: DtypeWarning: Columns (11,62,76,83,84,85,86,87,96,98,101,103,104,106,107,109,110,114) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("PrintingsData.csv",delimiter="|")


In [9]:
# Let's copy only a slice of the dataframe with the information we want
df_prints = df[['name','isReprint','uuid','identifiers.scryfallId','faceName','isFullArt']].copy()

In [12]:
df_prints.head()

,name,isReprint,uuid,identifiers.scryfallId,faceName,isFullArt
0,Plains,True,53532290-9194-5066-9318-b9cc09c52d8f,7ee52536-8cfa-482b-874e-094c0a081894,NaN,NaN
1,Island,True,4afd42fa-3e78-5e83-8320-dd5d13147038,161aaceb-d0bb-48c5-8bd4-bff321a94b2e,NaN,NaN
2,Swamp,True,3296e05b-7dae-5298-97a1-f33f3564ab88,4df49e68-cadf-4196-a3f4-ae38579edaeb,NaN,NaN
3,Mountain,True,83f5d717-4492-5ed7-9211-036b604524ad,890037ae-c366-4769-b7f7-7185a1bebca1,NaN,NaN
4,Forest,True,ab25e0c6-a0a0-56f7-9b17-efb5797d389a,86dae285-a59c-426c-b6cd-3683abea75a3,NaN,NaN


In [10]:
# For all rows, whenever faceName is null (whenever the card isn't double sided), let's have faceName = name, then drop the 'name' column
df_prints.loc[df_prints['faceName'].isnull(), 'faceName'] = df_prints['name']
df_prints.drop('name',axis=1,inplace=True)

In [11]:
# Let's keep only the printing info of the cards that are present in our 'DataCleansed.csv' file
df_prints = df_prints[df_prints['faceName'].isin(names)]

In [12]:
# We don't want cards that have FullArt, because then the image and the text will be mixed together, 
# and that isn't ideal for image recognition
df_prints = df_prints[df_prints['isFullArt'].isnull()]

In [13]:
# The same card might have been reprinted multiple times as well, so we need to drop duplicates
df_prints = df_prints.drop_duplicates(subset=['faceName'])

In [14]:
# We also don't need some of the columns anymore
df_prints = df_prints[['identifiers.scryfallId','faceName']]
df_prints = df_prints.rename(columns={"identifiers.scryfallId": "scryfallId", "faceName": "Name"})

In [18]:
df_prints

,scryfallId,Name
0,7ee52536-8cfa-482b-874e-094c0a081894,Plains
1,161aaceb-d0bb-48c5-8bd4-bff321a94b2e,Island
2,4df49e68-cadf-4196-a3f4-ae38579edaeb,Swamp
3,890037ae-c366-4769-b7f7-7185a1bebca1,Mountain
4,86dae285-a59c-426c-b6cd-3683abea75a3,Forest
...,...,...
67803,e65356e6-0ead-49fd-b069-be1ea9b1c105,Cathedral of Serra
67806,314fd1d7-4bd8-4d95-b7c2-1aa6660ab88a,Mountain Stronghold
67808,66641d88-b3f0-4bcd-8d2d-29aa2de69e30,Seafarer's Quay
67810,d43c01b7-443d-4061-a934-6863d230c9b8,Tolaria


In [19]:
# Let's check if the number of unique cards in our 'DataCleansed.csv' file is equal to the number of cards for which we have an image:
print(len(df_prints),len(names))

23919 23924


In [20]:
# 5 cards are missing!!
df_atomic[~df_atomic['Name'].isin(list(df_prints['Name']))]

,Color,Subtype,Text,Type,Loyalty,Power,Toughness,Keyword,Name,ManaValue
2515,['Red'],"['Nomad', 'Giant']",NaN,['Creature'],NaN,4,4,['None'],Fomori Nomad,5.0
7431,['Blue'],['Illusion'],NaN,['Creature'],NaN,2,3,['None'],Blind Phantasm,3.0
10880,['Red'],['Elemental'],"{R}, {T}, Sacrifice Flame Elemental: It deals ...",['Creature'],NaN,3,2,['None'],Flame Elemental,4.0
15345,['Green'],"['Cat', 'Monkey']",{2}{G}: Wily Bandar gains indestructible until...,['Creature'],NaN,1,1,['None'],Wily Bandar,1.0
21880,['Red'],"['Human', 'Barbarian']","{2}{R}, Sacrifice Barbarian Lunatic: It deals ...",['Creature'],NaN,2,1,['None'],Barbarian Lunatic,3.0


In [15]:
# Why are they missing?

missing_cards = list(df_atomic[~df_atomic['Name'].isin(list(df_prints['Name']))]['Name'])

for card in missing_cards:
    r = df[df['name']==card]['isFullArt']
    print(card)
    print(r)
    print('\n')

Fomori Nomad
3604     True
12896    True
Name: isFullArt, dtype: object


Blind Phantasm
12831    True
Name: isFullArt, dtype: object


Flame Elemental
Series([], Name: isFullArt, dtype: object)


Wily Bandar
Series([], Name: isFullArt, dtype: object)


Barbarian Lunatic
Series([], Name: isFullArt, dtype: object)




In [22]:
# Turns out that 3 of the cards were missing from the 'AllPrinting.json' files and 2 cards dont have a non full art version!
# Let's not forget to drop those 5 cards when we're working with the atomic data in 'DataCleansed.csv' later!

In [16]:
names_ids = list(df_prints[["Name","scryfallId"]].to_records(index=False))
names_ids

[('Plains', '7ee52536-8cfa-482b-874e-094c0a081894'),
 ('Island', '161aaceb-d0bb-48c5-8bd4-bff321a94b2e'),
 ('Swamp', '4df49e68-cadf-4196-a3f4-ae38579edaeb'),
 ('Mountain', '890037ae-c366-4769-b7f7-7185a1bebca1'),
 ('Forest', '86dae285-a59c-426c-b6cd-3683abea75a3'),
 ('Ajani, Caller of the Pride', '33e44e04-8330-49ca-906c-bb9bf0bc84ce'),
 ("Ajani's Chosen", '583bfbc1-638b-4de5-b865-0b00a69dd073'),
 ('Angelic Accord', '03f26bc2-53d7-4448-8021-de35aa82fcc6'),
 ('Angelic Wall', '7fe8f88e-8a51-494b-a008-fbfe624f97f7'),
 ('Archangel of Thune', '531cba81-afd7-4be4-adec-87edb77ba2a9'),
 ('Auramancer', '0a3dc4ab-1c45-4495-91b6-27d62087380c'),
 ('Banisher Priest', '06823bf8-2fca-49e1-ba40-9b61c9ae55b3'),
 ('Blessing', '4273cf50-db65-4b1f-95e1-f24ba6582c8b'),
 ('Bonescythe Sliver', 'a26bb68b-1830-470a-8cea-91edc7db0c57'),
 ('Brave the Elements', '097d7838-ae58-4306-ba0f-e914601b31b6'),
 ('Capashen Knight', '78802af4-46b5-4bac-8cdf-5b77d0b19895'),
 ('Celestial Flare', '6c8d1320-0f1a-4c66-86c9-9f8d

In [19]:
def get_imgUrl(names_ids):
    '''
    Uses the ScryfallID to call the Scryfall API and get the image URLs of every card in a dataframe.

            Parameters:
                    names_ids (list): A list of tuples with one (cardname,scryfallid) pair for each card.

            Returns:
                    imgUrls (list): A list of tuples with one (cardname,scryfallid,image url) pair for each card.
    '''    
    imgUrls = []

    for (name,id) in names_ids:
        call_url = "https://api.scryfall.com/cards/"+id
        call_card_img = requests.get(call_url)
        card_img_dict = json.loads(call_card_img.text)
        
        if 'card_faces' in card_img_dict.keys(): #If the card has 2 faces
            for n in range(0,2): #get me only the url of the face that corresponds to the Name of the card i want
                if card_img_dict['card_faces'][n]['name'] == id:
                    url_img = card_img_dict['card_faces'][n]['image_uris']['large'].split('?')[0]
        else: #If the card has 1 face, no problem
            url_img = card_img_dict['image_uris']['large'].split('?')[0]
        
        imgUrls.append((name,id,url_img))

    return imgUrls

# Call function
imgUrls = get_imgUrl(names_ids)
# Create new dataframe with cardname, scryfallid, image url columns
df_urls = pd.DataFrame(imgUrls, columns =['Name', 'scryfallId', 'URL'])

In [30]:
df_urls.head()

,Name,scryfallId,URL
0,Plains,7ee52536-8cfa-482b-874e-094c0a081894,https://c1.scryfall.com/file/scryfall-cards/la...
1,Island,161aaceb-d0bb-48c5-8bd4-bff321a94b2e,https://c1.scryfall.com/file/scryfall-cards/la...
2,Swamp,4df49e68-cadf-4196-a3f4-ae38579edaeb,https://c1.scryfall.com/file/scryfall-cards/la...
3,Mountain,890037ae-c366-4769-b7f7-7185a1bebca1,https://c1.scryfall.com/file/scryfall-cards/la...
4,Forest,86dae285-a59c-426c-b6cd-3683abea75a3,https://c1.scryfall.com/file/scryfall-cards/la...


In [31]:
len(df_urls)

23919

In [35]:
df_urls.to_csv("ImageURLs.csv","|",index=False)